# Assignment 2: NLP

## Envirnment Setup

Importing the dataset (needs to be done only once per notebook)

In [1]:
# needs to be run only once per notebook
# !wget "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
# !tar -xzf aclImdb_v1.tar.gz

In [80]:
!cat aclImdb/imdb.vocab | wc -l # number of vocab words

89526


## Imports

In [97]:
%reset
import nltk # natural language tool kit: for text pre-processing
import os # for listing directories
from bs4 import BeautifulSoup as bs # library for removing html tags from text
import numpy as np # no comment :P
from nltk.corpus import stopwords # a set of common stopwords from nltk
from gensim import models
import gensim
from collections import namedtuple
from sklearn.linear_model import LogisticRegression

Once deleted, variables cannot be recovered. Proceed (y/[n])? n
Nothing done.


In [82]:
# download resources for nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/abdelrahman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/abdelrahman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/abdelrahman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/abdelrahman/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [83]:
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', '``', "''", '...'])

## Reading the dataset and preprocessing the text

In [84]:
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def penn_to_wn(tag):
    return get_wordnet_pos(tag)

In [85]:
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

def get_bigrams(train_data):
    for docs in train_data: 
        bigram_finder = BigramCollocationFinder.from_words(docs)
        bigrams = bigram_finder.nbest(BigramAssocMeasures.chi_sq, 500)
        tokens = []
        for bigram_tuple in bigrams:
            x = "%s %s" % bigram_tuple
            tokens.append(x)
    print(tokens)

In [86]:
def read_data(data_set_path):
    data = []
    
    if any("all.txt" in s for s in os.listdir(data_set_path)):
        # read the reviews line by line
        all_data_file_path = os.path.join(data_set_path, 'all.txt')
        all_data_file = open(all_data_file_path, 'r')
        for line in all_data_file.readlines():
            vec = line.split(" ")
            vec = [item for item in vec if item != '\n']
            data.append(vec)
        return data
        
    
    for file_name in os.listdir(data_set_path):
        file_path = os.path.join(data_set_path, file_name)
        file = open(file_path, 'r')

        # read raw text
        text = file.read()

        # remove html tags
        text = bs(text, "html.parser").get_text()

        # tokenize the text
        tokens = nltk.word_tokenize(text)

        # get the part of speech tags for the tokens
        # This gives each token a tag from [NOUN, VERB, ADJ, ADV] which are
        # used by the lemmatizer to correctly lemmatize a word
        tagged_text = nltk.pos_tag(tokens)

        # lowercase, remove stop words, and lemmatize
        tokens = [wnl.lemmatize(tok.lower(), penn_to_wn(tag)) for (tok, tag) in tagged_text if tok.lower() not in stop_words and not tok.isdigit()]
        data.append(tokens)
    
    all_data_file_path = os.path.join(data_set_path, 'all.txt')
    all_data_file = open(all_data_file_path, 'w+')
    all_data_file.writelines(["%s\n" % " ".join(str(x) for x in item) for item in data])

    return data

In [87]:
train_pos = []
train_neg = []

wnl = nltk.WordNetLemmatizer()

In [88]:
train_pos = read_data('aclImdb/train/pos')
print("read {} positive training reviews".format(len(train_pos)))

train_neg = read_data('aclImdb/train/neg')
print("read {} negative training reviews".format(len(train_neg)))

train_data = train_pos + train_neg
print(len(train_data))

read 12500 positive training reviews
read 12500 negative training reviews
25000


In [89]:
test_pos = read_data('aclImdb/test/pos')
print("read {} positive test reviews".format(len(test_pos)))

test_neg = read_data('aclImdb/test/neg')
print("read {} negative test reviews".format(len(test_neg)))

test_data = test_pos + test_neg
print(len(test_data))

read 12500 positive test reviews
read 12500 negative test reviews
25000


## TF-IDF

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [91]:
# put positive and negative training data in one file
!cat aclImdb/train/pos/all.txt aclImdb/train/neg/all.txt > all_train.txt
!cat aclImdb/test/pos/all.txt aclImdb/test/neg/all.txt > all_test.txt

In [92]:
tfidf = TfidfVectorizer()
all_train_file = open('all_train.txt', 'r')
train_doc_matrix = tfidf.fit_transform([review for review in all_train_file.readlines()])

In [93]:
print(len(tfidf.vocabulary_))

67109


In [94]:
all_test_file = open('all_test.txt', 'r')
test_doc_matrix = tfidf.transform([review for review in all_test_file.readlines()])

In [95]:
print(test_doc_matrix.shape)

(25000, 67109)


In [98]:
clf3 = LogisticRegression()
clf3.fit(train_doc_matrix, [1]*12500 + [0]*12500)
print(clf3.score(test_doc_matrix, [1]*12500 + [0]*12500))

0.87792


In [99]:
word_idfs = dict(zip(tfidf.get_feature_names(), tfidf._tfidf.idf_))

## Word2Vec

In [100]:
from gensim.models import Word2Vec

In [101]:
word_dimensions = 100

model=Word2Vec(train_data, size = word_dimensions)

In [102]:
model.wv['cartoon']

array([-0.8168287 ,  0.73005927, -0.7006438 , -0.2594654 , -0.7625088 ,
       -1.6779494 , -0.83240485,  0.36481312, -1.245421  , -0.24876046,
       -0.70107067, -0.42921913, -0.9211775 , -1.3528693 ,  0.3654857 ,
        0.25650272,  0.64159644,  1.4829522 , -0.3695635 ,  1.3956082 ,
       -0.76010615,  0.07011928,  0.6077176 ,  0.3991185 , -0.06508129,
       -0.61021435, -0.34178463, -0.50792736,  0.15817441, -0.04236006,
        0.47983953,  0.21792863, -1.3419082 , -0.01015196, -0.86063254,
        1.1498944 ,  0.9942062 , -0.44470575,  0.21403773,  1.5123991 ,
        0.6447842 ,  1.1431347 , -0.48512664, -1.5760506 ,  0.7318202 ,
        0.2948373 , -0.5261131 , -0.27325976,  0.3320763 ,  0.3381813 ,
       -0.6799872 ,  0.57244056,  0.07642704, -1.8351017 , -0.24186401,
        1.0925725 ,  0.628451  , -0.12998264,  0.94145787, -0.05876609,
        0.36804083, -1.0539775 , -0.0313226 ,  0.1833193 ,  0.41587323,
       -0.25128555, -0.23727725, -0.81691056, -0.6709959 , -0.24

In [104]:
def get_doc_vecs_for_data(data, word_embeddings, word_weights = None, dimensions = word_dimensions):
    doc_vecs = np.empty([len(data), dimensions])

    for idx,review in enumerate(data):
        doc_vec = np.zeros([1,dimensions])
        for word in review:
            if word_weights != None:
                if word in word_embeddings and word in word_weights:
                    doc_vec += word_weights[word]*word_embeddings[word]
            else:
                if word in word_embeddings:
                    doc_vec += word_embeddings[word]
        doc_vecs[idx] = doc_vec/len(review)
    
    return doc_vecs

In [105]:
train_doc_vecs = get_doc_vecs_for_data(train_data, model.wv, word_weights=word_idfs, dimensions=word_dimensions)
test_doc_vecs = get_doc_vecs_for_data(test_data, model.wv, word_weights=word_idfs, dimensions=word_dimensions)

## Classifying using IDF weighted embeddings

In [106]:
clf = LogisticRegression()

clf.fit(train_doc_vecs, [1]*12500 + [0]*12500)
clf.score(test_doc_vecs, [1]*12500 + [0]*12500)

0.80628

## Using pretrained GloVe Embeddings

In [107]:
def load_glove_dict(size):
    assert size == 50 or size == 100 or size == 200 or size == 300
    
    glove_file_path = "glove.6B/glove.6B." + str(size) + "d.txt"
    
    glove_file = open(glove_file_path, 'r')
    
    glove_dict = {}
    for line in glove_file.readlines():
        tokens = line.split(" ")
        word = tokens[0]
        vec = [float(num) for num in tokens[1::]]
        glove_dict[word] = vec
    return glove_dict

In [113]:
glove_dict_50 = load_glove_dict(50)
glove_dict_300 = load_glove_dict(300)

In [114]:
glove_train_doc_vecs = get_doc_vecs_for_data(train_data, glove_dict_300, dimensions=300)
glove_test_doc_vecs = get_doc_vecs_for_data(test_data, glove_dict_300, dimensions=300)

In [115]:
%reset_selective -f glove_dict_50
%reset_selective -f glove_dict_300

In [116]:
clf2 = LogisticRegression()

clf2.fit(glove_train_doc_vecs, [1]*12500 + [0]*12500)
clf2.score(glove_test_doc_vecs, [1]*12500 + [0]*12500)

0.82624